# Working with Images stored on the OMERO server using Cell Profiler

This notebook demonstrates how to retrieve Images stored in OMERO and process them using [CellProfiler](http://cellprofiler.org/). The output is saved back to OMERO as CSV attachments.
For this example, we use the pipeline [FruitFlyCells](http://cellprofiler.org/examples/#FruitFlyCells).

In [ ]:
# When running the notebook on M1,
# you might have to remove this cell after removing the comment
#import os
#os.environ["JAVA_HOME"]="/full/path/to/conda_env"

### Import Packages

In [1]:
# %tb

# Import Cell Profiler Dependencies
import cellprofiler
import cellprofiler_core.preferences as cpprefs
import cellprofiler.modules as cpm
import cellprofiler_core.pipeline as cpp
cpprefs.set_headless()

# Inject Image module used to inject OMERO image planes into Cell Profiler Pipeline
from cellprofiler_core.modules.injectimage import InjectImage

# Import OMERO Python BlitzGateway
from omero.gateway import BlitzGateway

# Import Numpy
import numpy as np

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

from getpass import getpass

### Set Cell Output Directory

In [2]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### OMERO Credentials

In [3]:
HOST = 'wss://workshop.openmicroscopy.org/omero-ws'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, secure=True)
conn.connect()

Username: trainer-1
OMERO Password: ········


True

### Fetch OMERO Dataset that contains the Images to be analysed

In [4]:
# To be modified
# Please fill in the datasetId for the dataset named 'CellProfiler images'
datasetid = 1996
dataset = conn.getObject("Dataset", datasetid)
print('Dataset Name: ',dataset.getName())

Dataset Name:  CellProfiler images


### Load pipeline and inspect modules

In [5]:
pipeline = cpp.Pipeline()
pipeline.load("./pipelines/ExampleFly.cppipe")

# Remove first 4 modules: Images, Metadata, NamesAndTypes, Groups...
# (replaced by InjectImage module below)
for i in range(4):
    print('Remove module: ', pipeline.modules()[0].module_name)
    pipeline.remove_module(1)
print('Pipeline modules:')
for module in pipeline.modules():
    print(module.module_num, module.module_name)

Remove module:  Images
Remove module:  Metadata
Remove module:  NamesAndTypes
Remove module:  Groups
Pipeline modules:
1 Crop
2 Crop
3 Crop
4 IdentifyPrimaryObjects
5 IdentifySecondaryObjects
6 IdentifyTertiaryObjects
7 MeasureObjectSizeShape
8 MeasureObjectIntensity
9 MeasureTexture
10 MeasureObjectNeighbors
11 MeasureColocalization
12 MeasureImageIntensity
13 MeasureImageQuality
14 CalculateMath
15 ClassifyObjects
16 GrayToColor
17 ExportToSpreadsheet


### Run Cell Profiler Pipeline on the OMERO Images

In [6]:
warnings.filterwarnings('ignore')

Nuclei = pandas.DataFrame()
Cells = pandas.DataFrame()
Cytoplasm = pandas.DataFrame()
Image = pandas.DataFrame()

for image in dataset.listChildren():

    pixels = image.getPrimaryPixels()
    size_c = image.getSizeC()
    # For each Image in OMERO, we copy pipeline and inject image modules
    pipeline_copy = pipeline.copy()
    for c in range(0, size_c):

        plane = pixels.getPlane(0, c, 0)
        image_name = image.getName()

        # Name of the channel expected in the pipeline
        if c == 0:
            Image_Name = 'OrigBlue'
        if c == 1:
            Image_Name = 'OrigGreen'
        if c == 2:
            Image_Name = 'OrigRed'

        inject_image_module = InjectImage(Image_Name, plane)
        inject_image_module.set_module_num(1)
        pipeline_copy.add_module(inject_image_module)

    m = pipeline_copy.run()

    # Results obtained as CSV from Cell Profiler
    nuc_csv = pandas.read_csv(new_output_directory + '/Nuclei.csv')
    nuc_csv.ImageNumber = image.getName()
    Nuclei = Nuclei.append(nuc_csv)

    cel_csv = pandas.read_csv(new_output_directory + '/Cells.csv')
    cel_csv.ImageNumber = image.getName()
    Cells = Cells.append(cel_csv)

    cyto_csv = pandas.read_csv(new_output_directory + '/Cytoplasm.csv')
    cyto_csv.ImageNumber = image.getName()
    Cytoplasm = Cytoplasm.append(cyto_csv)

    image_csv = pandas.read_csv(new_output_directory + '/Image.csv')
    image_csv.ImageNumber = image.getName()
    Image = Image.append(image_csv)

## Results obtained as CSV from Cell Profiler

### Measurements (Nuclei)

In [7]:
Nuclei.describe()

,ObjectNumber,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Compactness,...,Texture_SumEntropy_CropBlue_3_02_256,Texture_SumEntropy_CropBlue_3_03_256,Texture_SumVariance_CropBlue_3_00_256,Texture_SumVariance_CropBlue_3_01_256,Texture_SumVariance_CropBlue_3_02_256,Texture_SumVariance_CropBlue_3_03_256,Texture_Variance_CropBlue_3_00_256,Texture_Variance_CropBlue_3_01_256,Texture_Variance_CropBlue_3_02_256,Texture_Variance_CropBlue_3_03_256
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,...,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,14.604938,363.395062,528.148148,110.209877,110.679012,87.654321,88.308642,98.362683,99.053594,1.209236,...,6.276824,6.161123,892.579905,781.800635,889.710470,771.259459,291.134343,301.072501,290.559905,299.840607
std,8.808631,167.526541,259.492394,50.365592,48.336018,50.580174,48.113574,50.420303,48.060805,0.241084,...,0.477060,0.516688,536.286191,500.166456,534.400875,473.379222,165.229027,172.589676,164.579625,172.491674
min,1.000000,102.000000,143.000000,24.000000,17.000000,3.000000,1.000000,13.504559,8.347619,0.980435,...,5.008854,4.840403,117.948218,80.923733,118.872358,89.796735,47.364895,48.467600,48.327980,51.749184
25%,7.000000,255.000000,374.000000,65.000000,73.000000,42.000000,44.000000,53.062670,59.897929,1.045232,...,5.962595,5.864263,425.304194,329.727529,424.779671,395.742859,147.399090,157.509572,147.591020,152.493029
50%,14.000000,349.000000,462.000000,110.000000,110.000000,84.000000,85.000000,96.686801,96.704370,1.111874,...,6.398197,6.282041,818.267349,697.500932,802.476970,687.262284,266.793135,274.726206,265.969167,274.988604
75%,21.000000,421.000000,648.000000,153.000000,151.000000,133.000000,130.000000,140.400978,138.813333,1.312468,...,6.614044,6.502537,1232.357438,1068.737487,1257.624165,1018.964844,382.713517,390.500486,377.983875,387.732947
max,34.000000,915.000000,1440.000000,196.000000,197.000000,173.000000,185.000000,182.951923,190.140351,2.303786,...,7.102833,7.037393,2282.288567,2133.603479,2300.416724,2086.358589,739.546537,762.344844,733.334349,765.037514


### Measurements (Cells)

In [8]:
Cells.head()

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,...,Texture_SumEntropy_CropBlue_3_02_256,Texture_SumEntropy_CropBlue_3_03_256,Texture_SumVariance_CropBlue_3_00_256,Texture_SumVariance_CropBlue_3_01_256,Texture_SumVariance_CropBlue_3_02_256,Texture_SumVariance_CropBlue_3_03_256,Texture_Variance_CropBlue_3_00_256,Texture_Variance_CropBlue_3_01_256,Texture_Variance_CropBlue_3_02_256,Texture_Variance_CropBlue_3_03_256
0,POS002.pattern,1,372,459,149,37,122,20,135.282258,28.174731,...,6.386346,6.350746,751.956406,501.186523,633.977067,640.784132,219.438478,222.919307,219.703030,219.534215
1,POS002.pattern,2,333,460,98,60,78,37,87.207207,47.597598,...,6.507063,6.502537,831.599848,483.017698,847.022904,878.240345,303.378182,312.699600,305.388510,300.244349
2,POS002.pattern,3,295,840,42,58,12,30,25.416949,45.142373,...,6.122834,6.083536,618.227747,629.807256,649.343906,510.917974,204.877481,216.603600,204.353133,196.324790
3,POS002.pattern,4,616,1426,46,77,0,46,16.006494,62.137987,...,6.454883,6.432906,1737.396018,1460.659070,1747.303198,1637.138785,498.449005,476.449335,498.497348,511.700145
4,POS002.pattern,5,394,816,52,84,28,50,39.335025,68.941624,...,6.062325,5.923365,805.153007,678.073029,876.803704,890.493326,241.351797,236.154948,240.267055,255.219621


### Measurements (Cytoplasm)

In [9]:
Cytoplasm.head()

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,...,Texture_SumEntropy_CropBlue_3_02_256,Texture_SumEntropy_CropBlue_3_03_256,Texture_SumVariance_CropBlue_3_00_256,Texture_SumVariance_CropBlue_3_01_256,Texture_SumVariance_CropBlue_3_02_256,Texture_SumVariance_CropBlue_3_03_256,Texture_Variance_CropBlue_3_00_256,Texture_Variance_CropBlue_3_01_256,Texture_Variance_CropBlue_3_02_256,Texture_Variance_CropBlue_3_03_256
0,POS002.pattern,1,84,459,149,37,122,20,135.083333,28.273810,...,2.584963,2.155639,10.499055,28.560000,40.472222,16.250000,5.722590,18.240000,13.076389,7.062500
1,POS002.pattern,2,82,460,98,60,78,37,87.365854,47.695122,...,3.077820,2.725481,24.000000,52.160000,23.812500,21.135802,9.950000,34.440000,14.609375,8.311728
2,POS002.pattern,3,156,840,42,58,12,30,26.166667,43.993590,...,4.532678,4.734184,132.877320,111.884444,156.550604,176.490000,49.464011,39.232222,52.479488,69.035000
3,POS002.pattern,4,367,1426,46,77,0,46,17.653951,59.831063,...,4.516527,4.279438,108.674375,111.608237,98.175239,99.405873,37.449844,48.125420,33.446871,36.670387
4,POS002.pattern,5,248,816,52,84,28,50,38.483871,66.616935,...,4.448740,4.171002,100.750230,80.890292,92.675101,91.911548,35.506885,38.528753,29.704982,28.522005


### Images (Summary)

In [10]:
Image.head()

,Classify_Large_NumObjectsPerBin,Classify_Large_PctObjectsPerBin,Classify_Medium_NumObjectsPerBin,Classify_Medium_PctObjectsPerBin,Classify_Small_NumObjectsPerBin,Classify_Small_PctObjectsPerBin,Correlation_Correlation_CropBlue_CropGreen,Correlation_Costes_CropBlue_CropGreen,Correlation_Costes_CropGreen_CropBlue,Correlation_K_CropBlue_CropGreen,...,Texture_Variance_CropBlue_3_02_256,Texture_Variance_CropBlue_3_03_256,Threshold_FinalThreshold_Cells,Threshold_FinalThreshold_Nuclei,Threshold_OrigThreshold_Cells,Threshold_OrigThreshold_Nuclei,Threshold_SumOfEntropies_Cells,Threshold_SumOfEntropies_Nuclei,Threshold_WeightedVariance_Cells,Threshold_WeightedVariance_Nuclei
0,3,8.823529,3,8.823529,9,26.470588,0.526074,0.990753,0.954735,0.943040,...,369.211218,372.200473,0.113534,0.054102,0.113534,0.054102,-13.206373,-12.888235,0.636118,1.264639
0,2,7.407407,2,7.407407,9,33.333333,0.178739,0.621968,0.428216,0.446542,...,550.550634,549.096159,0.086443,0.070660,0.086443,0.070660,-12.715738,-12.687216,0.401940,0.931757
0,0,0.000000,1,5.000000,8,40.000000,0.382287,0.941827,0.944726,0.416789,...,314.672127,315.466506,0.062577,0.050299,0.062577,0.050299,-12.997775,-13.107301,0.285742,1.004763


### Push Results back to OMERO

In [12]:
import glob

for file_to_upload in glob.glob(new_output_directory + '/*.csv'):

    if file_to_upload.endswith('Nuclei.csv'):
        Nuclei.to_csv(file_to_upload)
    elif file_to_upload.endswith('Cells.csv'):
        Cells.to_csv(file_to_upload)
    elif file_to_upload.endswith('Cytoplasm.csv'):
        Cytoplasm.to_csv(file_to_upload)
    elif file_to_upload.endswith('Image.csv'):
        Image.to_csv(file_to_upload)

    # create the original file and file annotation (uploads the file etc.)
    namespace = "training.meeting.demo"
    print("\nCreating an OriginalFile and FileAnnotation")
    file_ann = conn.createFileAnnfromLocalFile(file_to_upload, mimetype="text/plain", ns=namespace, desc=None)
    print("Attaching FileAnnotation to Dataset: ", "File ID:", file_ann.getId(),
        ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize())
    dataset.linkAnnotation(file_ann)


Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 135224 , Image.csv Size: 54496

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 135225 , Nuclei.csv Size: 207531

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 135226 , Cytoplasm.csv Size: 185348

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 135227 , Cells.csv Size: 187106


#### Close the connection to the OMERO server

In [13]:
conn.close()

### License (BSD 2-Clause)
Copyright (C) 2020-2023 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.